### Tuning Supervised Learners

#### Measures of Performance
##### Confusion matrix
Visualises the performance of a classification algorithm. Rows represent the items belonging to the actual classes, column represents the items belonging to the predicted classes.

Samples are categorised by being True/False Negatives/Positives (TN, FN, TP, FP). False positives are usually called a type 1 error, false negatives a type 2 error. There's usually a trade-off between the two, and which error you should be more concerned aout depends on the problem you're trying to solve.

##### Recall
A measure that indicates the ratio of positive test data items that are correctly identified out of all the items that are actually positive. Can be computed from the confusion matrix as $$Recall = \frac{TP}{TP + FN}$$

##### Precision
Measure that indicates ratio of the number of correctly predicted positive points to the number of all the points that were predicted as positive. $$Precision = \frac{TP}{TP + FP}$$


